In [ ]:
 # Packages used. After you run it once, you can comment it out
 !pip install haversine

In [ ]:
#Imports
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from haversine import haversine, Unit

# Preprocessing the Data

upload - Edges_new.csv, MTA_Subway_Monthly_Ridership__Feb2022_to_Jan2023.csv, flowInFlowOutSolution.csv

In [ ]:
# Getting the complex id, location, and stop_names used in the edges (Physical Network)
edges = pd.read_csv("Edges_new.csv")
edges.GTFS_Latitude = round(edges.GTFS_Latitude.astype("float"),5)
edges.GTFS_Longitude = round(edges.GTFS_Longitude.astype("float"),5)
edges['location'] = (edges[["GTFS_Latitude","GTFS_Longitude"]]).apply(tuple, axis = 1)
edges_join = edges[['Complex_ID', 'location','Stop_Name']].drop_duplicates()
edges_join['CID, Stop'] = edges_join[["Complex_ID","Stop_Name"]].apply(tuple, axis=1)
edges_join

,Complex_ID,location,Stop_Name,"CID, Stop"
0,2,"(40.77026, -73.91784)",Astoria Blvd,"(2, Astoria Blvd)"
1,3,"(40.76678, -73.92148)",30 Av,"(3, 30 Av)"
2,4,"(40.76182, -73.92551)",Broadway,"(4, Broadway)"
3,5,"(40.7568, -73.92958)",36 Av,"(5, 36 Av)"
4,6,"(40.75288, -73.93276)",39 Av-Dutch Kills,"(6, 39 Av-Dutch Kills)"
...,...,...,...,...
460,517,"(40.52551, -74.20006)",Prince's Bay,"(517, Prince's Bay)"
461,518,"(40.52241, -74.21785)",Pleasant Plains,"(518, Pleasant Plains)"
462,519,"(40.51963, -74.22914)",Richmond Valley,"(519, Richmond Valley)"
463,522,"(40.51276, -74.25196)",Tottenville,"(522, Tottenville)"


In [ ]:
# Getting the station complex id, location used in the ridership (Origin Destination Network)

df = pd.read_csv('MTA_Subway_Monthly_Ridership__Feb2022_to_Jan2023.csv')
df.latitude = round(df.latitude.astype("float"),5)
df.longitude = round(df.longitude.astype("float"),5)
df['location'] = df[["latitude","longitude"]].apply(tuple, axis=1)
df_join = df[['station_complex_id', "location"]].drop_duplicates()
df_join

,station_complex_id,location
0,A002,"(40.76266, -73.96725)"
12,A006,"(40.76481, -73.97335)"
24,A010,"(40.76466, -73.98066)"
36,A013,"(40.7599, -73.98414)"
48,A022,"(40.74972, -73.98782)"
...,...,...
5063,R728,"(40.86953, -73.84638)"
5075,R729,"(40.87866, -73.83859)"
5087,R730,"(40.8883, -73.83083)"
5099,S101,"(40.64375, -74.07364)"


In [ ]:
# Finding the nearest stops to assign each station Complex ID to a nearest Stop (joining the two data frames)
stations_df = df_join
df_stops = edges_join
stations_df['Latitude'] = stations_df['location'].apply(lambda x: x[0])
stations_df['Longitude'] = stations_df['location'].apply(lambda x: x[1])
df_stops['Latitude'] = df_stops['location'].apply(lambda x: x[0])
df_stops['Longitude'] = df_stops['location'].apply(lambda x: x[1])

def find_nearest_stop(row, stops_df):
    distances = stops_df.apply(lambda stop: haversine((row['Latitude'], row['Longitude']),
                                                      (stop['Latitude'], stop['Longitude']),
                                                      unit=Unit.METERS), axis=1)
    nearest_stop_idx = distances.idxmin()
    return stops_df.loc[nearest_stop_idx, 'CID, Stop']

stations_df['Nearest_Stop'] = stations_df.apply(find_nearest_stop, axis=1, stops_df=df_stops)
stations_df

,station_complex_id,location,Latitude,Longitude,Nearest_Stop
0,A002,"(40.76266, -73.96725)",40.76266,-73.96725,"(613, Lexington Av/59 St)"
12,A006,"(40.76481, -73.97335)",40.76481,-73.97335,"(8, 5 Av/59 St)"
24,A010,"(40.76466, -73.98066)",40.76466,-73.98066,"(9, 57 St-7 Av)"
36,A013,"(40.7599, -73.98414)",40.75990,-73.98414,"(10, 49 St)"
48,A022,"(40.74972, -73.98782)",40.74972,-73.98782,"(607, 34 St-Herald Sq)"
...,...,...,...,...,...
5063,R728,"(40.86953, -73.84638)",40.86953,-73.84638,"(444, Gun Hill Rd)"
5075,R729,"(40.87866, -73.83859)",40.87866,-73.83859,"(443, Baychester Av)"
5087,R730,"(40.8883, -73.83083)",40.88830,-73.83083,"(443, Baychester Av)"
5099,S101,"(40.64375, -74.07364)",40.64375,-74.07364,"(502, Tompkinsville)"


In [ ]:
# merge all the info
dfFullAgg = df.groupby(["station_complex_id","borough","routes","latitude","longitude"])["ridership"].sum().reset_index()
ridershipFullInfo = pd.merge(stations_df, dfFullAgg).drop(columns = ['latitude','longitude'])
ridershipFullInfo

,station_complex_id,location,Latitude,Longitude,Nearest_Stop,borough,routes,ridership
0,A002,"(40.76266, -73.96725)",40.76266,-73.96725,"(613, Lexington Av/59 St)",M,"5,N,R,4,,W,6",10082907
1,A006,"(40.76481, -73.97335)",40.76481,-73.97335,"(8, 5 Av/59 St)",M,"N,R,W",3065515
2,A010,"(40.76466, -73.98066)",40.76466,-73.98066,"(9, 57 St-7 Av)",M,"N,R,Q,W",6579806
3,A013,"(40.7599, -73.98414)",40.75990,-73.98414,"(10, 49 St)",M,"N,R,W",4772207
4,A022,"(40.74972, -73.98782)",40.74972,-73.98782,"(607, 34 St-Herald Sq)",M,"F,M,N,R,Q,B,W,D",21372033
...,...,...,...,...,...,...,...,...
420,R728,"(40.86953, -73.84638)",40.86953,-73.84638,"(444, Gun Hill Rd)",BX,5,928838
421,R729,"(40.87866, -73.83859)",40.87866,-73.83859,"(443, Baychester Av)",BX,5,557105
422,R730,"(40.8883, -73.83083)",40.88830,-73.83083,"(443, Baychester Av)",BX,5,768803
423,S101,"(40.64375, -74.07364)",40.64375,-74.07364,"(502, Tompkinsville)",SI,SIR,1770469


In [ ]:
#finding which stations are physically connected

edges['From'] = edges[["Complex_ID","Stop_Name"]].apply(tuple, axis=1)
edges['To'] = edges[["Complex_ID_Next","Stop_Name_Next"]].apply(tuple, axis=1)
neighs = nx.from_pandas_edgelist(edges, source='From', target='To')
neighbors_dict = {node: list(neighs.neighbors(node)) for node in neighs.nodes}
neighbors_dict
## add it to full info df
ridershipFullInfo['Connections'] = ridershipFullInfo['Nearest_Stop'].apply(lambda x: neighbors_dict[x])
## i just dropped duplicates there were only 2 duplicates its not the best way but i dont think it matters since its just 2
ridershipFullInfo = ridershipFullInfo.drop_duplicates(subset='Nearest_Stop')
ridershipFullInfo.to_csv('ridershipFullInfo.csv')
ridershipFullInfo

,station_complex_id,location,Latitude,Longitude,Nearest_Stop,borough,routes,ridership,Connections
0,A002,"(40.76266, -73.96725)",40.76266,-73.96725,"(613, Lexington Av/59 St)",M,"5,N,R,4,,W,6",10082907,"[(6, 39 Av-Dutch Kills), (8, 5 Av/59 St)]"
1,A006,"(40.76481, -73.97335)",40.76481,-73.97335,"(8, 5 Av/59 St)",M,"N,R,W",3065515,"[(613, Lexington Av/59 St), (9, 57 St-7 Av)]"
2,A010,"(40.76466, -73.98066)",40.76466,-73.98066,"(9, 57 St-7 Av)",M,"N,R,Q,W",6579806,"[(8, 5 Av/59 St), (10, 49 St)]"
3,A013,"(40.7599, -73.98414)",40.75990,-73.98414,"(10, 49 St)",M,"N,R,W",4772207,"[(9, 57 St-7 Av), (611, Times Sq-42 St)]"
4,A022,"(40.74972, -73.98782)",40.74972,-73.98782,"(607, 34 St-Herald Sq)",M,"F,M,N,R,Q,B,W,D",21372033,"[(611, Times Sq-42 St), (13, 28 St), (609, 42 ..."
...,...,...,...,...,...,...,...,...,...
418,R726,"(40.85436, -73.8605)",40.85436,-73.86050,"(446, Morris Park)",BX,5,479759,"[(445, Pelham Pkwy)]"
419,R727,"(40.85899, -73.85536)",40.85899,-73.85536,"(445, Pelham Pkwy)",BX,5,557567,"[(444, Gun Hill Rd), (446, Morris Park)]"
420,R728,"(40.86953, -73.84638)",40.86953,-73.84638,"(444, Gun Hill Rd)",BX,5,928838,"[(443, Baychester Av), (445, Pelham Pkwy)]"
421,R729,"(40.87866, -73.83859)",40.87866,-73.83859,"(443, Baychester Av)",BX,5,557105,"[(442, Eastchester-Dyre Av), (444, Gun Hill Rd)]"


# Network Analysis

In [ ]:
from ast import literal_eval

flows = pd.read_csv('flowInFlowOutSolution.csv', converters={'to': literal_eval, 'from': literal_eval})
nodes = ridershipFullInfo

edges = pd.read_csv('Edges_new.csv')
def hav_dist(x):
  return haversine((x['GTFS_Latitude'], x['GTFS_Longitude']), (x['GTFS_Latitude_Next'], x['GTFS_Longitude_Next']), unit=Unit.METERS)

edges["Stop_Name"]=edges["Stop_Name"].str.strip()
edges["Stop_Name_Next"]=edges["Stop_Name_Next"].str.strip()

## have to figure out a way so that the nodes in the OD and PN are the same
edges['distance'] = edges.apply(hav_dist, axis = 1)
edges['from'] = edges[['Complex_ID', 'Stop_Name']].apply(tuple, axis = 1)
edges['to'] = edges[['Complex_ID_Next', 'Stop_Name_Next']].apply(tuple, axis = 1)
currentRidership = nodes['Nearest_Stop'].to_list()
PN_edges = edges[edges['from'].isin(currentRidership) & edges['to'].isin(currentRidership)]
currentPN = set(PN_edges['from'].to_list()) | set(PN_edges['to'].to_list())
nodes = nodes[nodes['Nearest_Stop'].isin(currentPN)]
currentRidership = nodes['Nearest_Stop'].to_list()
flows = flows[flows['from'].isin(currentRidership) & flows['to'].isin(currentRidership)]

In [ ]:
shortest_paths = dict(nx.all_pairs_shortest_path(gPN), weight = 'distance')

In [ ]:
connectivity = []
for i in shortest_paths:
  paths = shortest_paths[i]
  for j in paths:
    if i != j:
      u = i
      v = j
      node_list = paths[j]
      edges = []
      for k in range(len(node_list) - 1):
        edge = (node_list[k], node_list[k + 1])
        edges.append(edge)
      connectivity.append([u, v, edges])
connectivity_df = pd.DataFrame(connectivity, columns = ['from', 'to', 'edges'])
connectivity_df = connectivity_df.merge(flows)
PN_edges['edge'] = PN_edges[['from', 'to']].apply(tuple, axis = 1)






In [ ]:
PN_edges

,Unnamed: 0,Daytime_Routes,Complex_ID,Complex_ID_Next,GTFS_Latitude_Next,GTFS_Longitude_Next,GTFS_Latitude,GTFS_Longitude,Stop_Name,Stop_Name_Next,distance,from,to,edge,total_ridership
1,1,N W,3,2,40.770258,-73.917843,40.766779,-73.921479,30 Av,Astoria Blvd,493.366940,"(3, 30 Av)","(2, Astoria Blvd)","((3, 30 Av), (2, Astoria Blvd))",6.702221e+04
2,2,N W,4,3,40.766779,-73.921479,40.761820,-73.925508,Broadway,30 Av,647.455046,"(4, Broadway)","(3, 30 Av)","((4, Broadway), (3, 30 Av))",1.332698e+05
3,3,N W,5,4,40.761820,-73.925508,40.756804,-73.929575,36 Av,Broadway,654.543978,"(5, 36 Av)","(4, Broadway)","((5, 36 Av), (4, Broadway))",1.755904e+05
4,4,N W,6,5,40.756804,-73.929575,40.752882,-73.932755,39 Av-Dutch Kills,36 Av,511.796935,"(6, 39 Av-Dutch Kills)","(5, 36 Av)","((6, 39 Av-Dutch Kills), (5, 36 Av))",1.868389e+05
5,5,N R W,613,6,40.752882,-73.932755,40.762660,-73.967258,Lexington Av/59 St,39 Av-Dutch Kills,3102.837813,"(613, Lexington Av/59 St)","(6, 39 Av-Dutch Kills)","((613, Lexington Av/59 St), (6, 39 Av-Dutch Ki...",1.869945e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,438,7,606,461,40.750582,-73.940202,40.747023,-73.945264,Court Sq,Queensboro Plaza,581.760240,"(606, Court Sq)","(461, Queensboro Plaza)","((606, Court Sq), (461, Queensboro Plaza))",2.473093e+07
437,439,7,463,606,40.747023,-73.945264,40.742216,-73.948916,Hunters Point Av,Court Sq,616.734011,"(463, Hunters Point Av)","(606, Court Sq)","((463, Hunters Point Av), (606, Court Sq))",2.854118e+07
438,440,7,464,463,40.742216,-73.948916,40.742626,-73.953581,Vernon Blvd-Jackson Av,Hunters Point Av,395.648133,"(464, Vernon Blvd-Jackson Av)","(463, Hunters Point Av)","((464, Vernon Blvd-Jackson Av), (463, Hunters ...",2.896297e+07
439,441,7,610,464,40.742626,-73.953581,40.751431,-73.976041,Grand Central-42 St,Vernon Blvd-Jackson Av,2130.368440,"(610, Grand Central-42 St)","(464, Vernon Blvd-Jackson Av)","((610, Grand Central-42 St), (464, Vernon Blvd...",3.025646e+07


In [ ]:
PN_edges['total_ridership'] = PN_edges['edge'].apply(lambda edge:
    sum(connectivity_df[connectivity_df['edges'].apply(lambda paths: edge in paths)]['ridership'])
)

nodes.to_csv('node.csv')
flows.to_csv('OD_edges.csv')
PN_edges[['from', 'to', 'total_ridership', 'distance']].to_csv('PN_edges.csv')